In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
dataset = pd.read_csv("https://raw.githubusercontent.com/PersDep/data-mining-intro-2021/main/hw03-EDA-data/german_credit.csv")
dataset

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personal_status_sex,other_debtors,present_residence,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,credit_risk
0,no checking account,18,all credits at this bank paid back duly,car (used),1049,unknown/no savings account,< 1 yr,< 20,female : non-single or male : single,none,>= 7 yrs,car or other,21,none,for free,1,skilled employee/official,0 to 2,no,no,good
1,no checking account,9,all credits at this bank paid back duly,others,2799,unknown/no savings account,1 <= ... < 4 yrs,25 <= ... < 35,male : married/widowed,none,1 <= ... < 4 yrs,unknown / no property,36,none,for free,2-3,skilled employee/official,3 or more,no,no,good
2,... < 0 DM,12,no credits taken/all credits paid back duly,retraining,841,... < 100 DM,4 <= ... < 7 yrs,25 <= ... < 35,female : non-single or male : single,none,>= 7 yrs,unknown / no property,23,none,for free,1,unskilled - resident,0 to 2,no,no,good
3,no checking account,12,all credits at this bank paid back duly,others,2122,unknown/no savings account,1 <= ... < 4 yrs,20 <= ... < 25,male : married/widowed,none,1 <= ... < 4 yrs,unknown / no property,39,none,for free,2-3,unskilled - resident,3 or more,no,yes,good
4,no checking account,12,all credits at this bank paid back duly,others,2171,unknown/no savings account,1 <= ... < 4 yrs,< 20,male : married/widowed,none,>= 7 yrs,car or other,38,bank,rent,2-3,unskilled - resident,0 to 2,no,yes,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no checking account,24,no credits taken/all credits paid back duly,furniture/equipment,1987,unknown/no savings account,1 <= ... < 4 yrs,25 <= ... < 35,male : married/widowed,none,>= 7 yrs,unknown / no property,21,none,for free,1,unskilled - resident,3 or more,no,no,bad
996,no checking account,24,no credits taken/all credits paid back duly,others,2303,unknown/no savings account,>= 7 yrs,< 20,male : married/widowed,co-applicant,< 1 yr,unknown / no property,45,none,rent,1,skilled employee/official,0 to 2,no,no,bad
997,... >= 200 DM / salary for at least 1 year,21,all credits at this bank paid back duly,others,12680,... >= 1000 DM,>= 7 yrs,< 20,male : married/widowed,none,>= 7 yrs,real estate,30,none,own,1,manager/self-empl./highly qualif. employee,0 to 2,yes (under customer name),no,bad
998,... < 0 DM,12,no credits taken/all credits paid back duly,furniture/equipment,6468,... >= 1000 DM,unemployed,25 <= ... < 35,male : married/widowed,none,< 1 yr,real estate,52,none,rent,1,manager/self-empl./highly qualif. employee,0 to 2,yes (under customer name),no,bad


In [3]:
dataset.duplicated().value_counts()

False    1000
dtype: int64

In [4]:
dataset.isna().sum().sum()

0

In [5]:
dataset.info()
dataset.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   status                   1000 non-null   object
 1   duration                 1000 non-null   int64 
 2   credit_history           1000 non-null   object
 3   purpose                  1000 non-null   object
 4   amount                   1000 non-null   int64 
 5   savings                  1000 non-null   object
 6   employment_duration      1000 non-null   object
 7   installment_rate         1000 non-null   object
 8   personal_status_sex      1000 non-null   object
 9   other_debtors            1000 non-null   object
 10  present_residence        1000 non-null   object
 11  property                 1000 non-null   object
 12  age                      1000 non-null   int64 
 13  other_installment_plans  1000 non-null   object
 14  housing                  1000 non-null   

,duration,amount,age
count,1000.000000,1000.00000,1000.00000
mean,20.903000,3271.24800,35.54200
std,12.058814,2822.75176,11.35267
min,4.000000,250.00000,19.00000
25%,12.000000,1365.50000,27.00000
50%,18.000000,2319.50000,33.00000
75%,24.000000,3972.25000,42.00000
max,72.000000,18424.00000,75.00000


In [6]:
from sklearn.preprocessing import LabelEncoder
for x in dataset.columns:
  if dataset[x].dtype == 'object':
    dataset[x] = LabelEncoder().fit_transform(dataset[x])
dataset["age"] = pd.cut(dataset.age, (18, 25, 35, 50, 65, 80), labels=[0, 1, 2, 3, 4])
dataset["amount"] = pd.cut(dataset.amount, (250, 1000, 2000, 4000, 8000, 20000), labels=[0, 1, 2, 3, 4])
dataset["duration"] = pd.cut(dataset.duration, (4, 6, 8, 12, 18, 24, 36, 48, 72), labels=[0, 1, 2, 3, 4, 5, 6, 7])
dataset

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personal_status_sex,other_debtors,present_residence,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,credit_risk
0,3,3,0,2,1,4,2,2,0,2,3,1,0,1,0,0,1,0,0,0,1
1,3,2,0,5,2,4,0,1,3,2,0,3,2,1,0,1,1,1,0,0,1
2,0,2,4,8,0,0,1,1,0,2,3,3,0,1,0,0,3,0,0,0,1
3,3,2,0,5,2,4,0,0,3,2,0,3,2,1,0,1,3,1,0,1,1
4,3,2,0,5,2,4,0,2,3,2,3,1,2,0,2,1,3,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,4,4,4,1,4,0,1,3,2,3,3,0,1,0,0,3,1,0,0,0
996,3,4,4,5,2,4,3,2,3,0,2,3,2,1,2,0,1,0,0,0,0
997,1,4,0,5,4,1,3,2,3,2,3,2,1,1,1,0,0,0,1,0,0
998,0,2,4,4,3,1,4,1,3,2,2,2,3,1,2,0,0,0,1,0,0


### Feature Selection

In [7]:
X = dataset.drop(['credit_risk'], 1)
y = dataset['credit_risk']

In [8]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
fs = SelectKBest(score_func=f_classif, k='all')
fs.fit(X, y)

rate = pd.DataFrame(fs.scores_, X.columns, columns=['score'])
rate = rate.sort_values('score', axis=0)
rate

,score
people_liable,0.009071
employment_duration,0.079635
installment_rate,0.126847
present_residence,0.155848
amount,0.428326
other_debtors,0.807899
job,1.071934
telephone,1.328891
number_credits,2.091652
duration,2.588473


### Logistic Regression / SVM

In [9]:
cols = rate.index.values[::-1]
features = X[[col for col in cols[:len(cols)//2]]]

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=11)

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
def train_test_loop(model, Xtrain, ytrain, Xtest, ytest, lin=False):
  model.fit(Xtrain, ytrain)
  ypred = model.predict(Xtest)
  if lin:
    ypred = [(y > 0.5) for y in ypred]
  print(confusion_matrix(ytest, ypred))
  print(classification_report(ytest, ypred))

In [12]:
from sklearn.linear_model import LinearRegression
train_test_loop(LinearRegression(), X_train, y_train, X_test, y_test, True)

[[  6  57]
 [  4 133]]
              precision    recall  f1-score   support

           0       0.60      0.10      0.16        63
           1       0.70      0.97      0.81       137

    accuracy                           0.69       200
   macro avg       0.65      0.53      0.49       200
weighted avg       0.67      0.69      0.61       200



In [13]:
from sklearn.linear_model import LogisticRegression
train_test_loop(LogisticRegression(solver='liblinear'), X_train, y_train, X_test, y_test)

[[  7  56]
 [  5 132]]
              precision    recall  f1-score   support

           0       0.58      0.11      0.19        63
           1       0.70      0.96      0.81       137

    accuracy                           0.69       200
   macro avg       0.64      0.54      0.50       200
weighted avg       0.66      0.69      0.62       200



In [14]:
from sklearn.svm import SVC
train_test_loop(SVC(kernel="poly", degree=10), X_train, y_train, X_test, y_test)

[[ 30  33]
 [ 20 117]]
              precision    recall  f1-score   support

           0       0.60      0.48      0.53        63
           1       0.78      0.85      0.82       137

    accuracy                           0.73       200
   macro avg       0.69      0.67      0.67       200
weighted avg       0.72      0.73      0.73       200



### Logistic Regression / SVM with SMOTE

In [15]:
#!pip install smote-variants
from smote_variants import SMOTE
smote = SMOTE()

In [26]:
X_smote, y_smote = smote.sample(features.values, y.values)
print("Before SMOTE:")
y.value_counts()

2021-03-24 13:53:26,180:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")


Before SMOTE:


1    700
0    300
Name: credit_risk, dtype: int64

In [24]:
print("After SMOTE:")
np.unique(y_smote, return_counts=True)
print(X_smote.shape)
print(y_smote.shape)

After SMOTE:
(1200, 10)
(1200,)


In [18]:
X_train_smote, X_test_smote, y_train_smote, y_test_smote = train_test_split(X_smote, y_smote, test_size=0.2, random_state=110)

In [19]:
train_test_loop(LinearRegression(), X_train_smote, y_train_smote, X_test, y_test, True)

[[39 24]
 [43 94]]
              precision    recall  f1-score   support

           0       0.48      0.62      0.54        63
           1       0.80      0.69      0.74       137

    accuracy                           0.67       200
   macro avg       0.64      0.65      0.64       200
weighted avg       0.70      0.67      0.67       200



In [20]:
train_test_loop(LogisticRegression(solver='liblinear'), X_train_smote, y_train_smote, X_test, y_test)

[[38 25]
 [42 95]]
              precision    recall  f1-score   support

           0       0.47      0.60      0.53        63
           1       0.79      0.69      0.74       137

    accuracy                           0.67       200
   macro avg       0.63      0.65      0.64       200
weighted avg       0.69      0.67      0.67       200



In [21]:
train_test_loop(SVC(kernel="poly", degree=10), X_train_smote, y_train_smote, X_test, y_test)
train_test_loop(SVC(kernel="poly", degree=10), X_train_smote, y_train_smote, X_test_smote, y_test_smote)

[[ 58   5]
 [  5 132]]
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        63
           1       0.96      0.96      0.96       137

    accuracy                           0.95       200
   macro avg       0.94      0.94      0.94       200
weighted avg       0.95      0.95      0.95       200

[[116  26]
 [ 36 102]]
              precision    recall  f1-score   support

           0       0.76      0.82      0.79       142
           1       0.80      0.74      0.77       138

    accuracy                           0.78       280
   macro avg       0.78      0.78      0.78       280
weighted avg       0.78      0.78      0.78       280



In [27]:
for test_object in X_test.values:
  for i, x in enumerate(X_smote):
    if np.array_equal(x, test_object):
      X_smote = np.delete(X_smote, i, axis=0)
      y_smote = np.delete(y_smote, i)
      break
X_train_smote, y_train_smote = X_smote, y_smote
X_smote.shape, y_smote.shape

((1200, 10), (1200,))

In [28]:
train_test_loop(SVC(kernel="poly", degree=7), X_train_smote, y_train_smote, X_test, y_test)

[[ 47  16]
 [ 31 106]]
              precision    recall  f1-score   support

           0       0.60      0.75      0.67        63
           1       0.87      0.77      0.82       137

    accuracy                           0.77       200
   macro avg       0.74      0.76      0.74       200
weighted avg       0.78      0.77      0.77       200

